In [1]:
import plotly
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import statsmodels.api as sm
import pylab as py

plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams["font.size"] = "17"

from greykite.framework.templates.autogen.forecast_config import EvaluationPeriodParam
from greykite.framework.templates.autogen.forecast_config import ForecastConfig
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.autogen.forecast_config import ModelComponentsParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.algo.common.seasonality_inferrer import SeasonalityInferConfig
from greykite.algo.common.seasonality_inferrer import SeasonalityInferrer
from greykite.algo.common.seasonality_inferrer import TrendAdjustMethodEnum
from greykite.common import constants as cst
from plotly.offline import init_notebook_mode, iplot

import warnings
from collections import defaultdict
warnings.filterwarnings("ignore")

In [2]:
path_file = 'C:\\Users\\tayal\\OneDrive\\Desktop\\Product1.xlsx'

df = pd.read_excel(path_file)
Month = UnivariateTimeSeries()
Month.load_data(
     df=df,
     time_col="date",
     value_col="qty",
     freq="D")

In [3]:
print(Month.describe_time_col())
print(Month.describe_value_col())
print(df.head())

{'data_points': 10227, 'mean_increment_secs': 86400.0, 'min_timestamp': Timestamp('1996-01-01 00:00:00'), 'max_timestamp': Timestamp('2023-12-31 00:00:00')}
count    10227.000000
mean       251.400900
std        144.608247
min          0.000000
25%        124.000000
50%        253.000000
75%        376.000000
max        500.000000
Name: y, dtype: float64
        date  qty
0 1996-01-01   99
1 1996-01-02   20
2 1996-01-03  339
3 1996-01-04   15
4 1996-01-05  261


In [4]:
print(df.tail())
print(df.info())
print(df.describe().T)

            date  qty
10222 2023-12-27  317
10223 2023-12-28   34
10224 2023-12-29  421
10225 2023-12-30  266
10226 2023-12-31  493
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10227 entries, 0 to 10226
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    10227 non-null  datetime64[ns]
 1   qty     10227 non-null  int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 159.9 KB
None
       count      mean         std  min    25%    50%    75%    max
qty  10227.0  251.4009  144.608247  0.0  124.0  253.0  376.0  500.0


In [5]:
fig = Month.plot()
iplot(fig)

# Normal Code

In [6]:
forecast_horizon = 90
meta_data_params = MetadataParam(
     time_col="date",
     value_col="qty",
     freq="D",
 )

In [7]:
cv_min_train_periods = 821
# Let CV use most recent splits for cross-validation.
cv_use_most_recent_splits = True
# Determine the maximum number of validations.
cv_max_splits = 657
evaluation_period_param = EvaluationPeriodParam(
     test_horizon=forecast_horizon,
     cv_horizon=forecast_horizon,
     periods_between_train_test=0,
     cv_min_train_periods=cv_min_train_periods,
     cv_expanding_window=False,
     cv_use_most_recent_splits=cv_use_most_recent_splits,
     cv_periods_between_splits=None,
     cv_periods_between_train_test=0,
     cv_max_splits=cv_max_splits,
)

In [8]:
extra_pred_cols = ["ct1"]
autoregression = None

# Specify the model parameters
model_components = ModelComponentsParam(
     growth=dict(growth_term=["linear"]),
     seasonality=dict(
         yearly_seasonality=['auto'],
         quarterly_seasonality=['auto'],
         monthly_seasonality=['auto'],
         weekly_seasonality=[False],
         daily_seasonality=[False]
     ),
     custom=dict(
         fit_algorithm_dict=dict(fit_algorithm="lasso"),
         extra_pred_cols=extra_pred_cols
     ),
     regressors=dict(regressor_cols=None),
     autoregression=autoregression,
     uncertainty=dict(uncertainty_dict={
        "uncertainty_method": "simple_conditional_residuals",
        "params": {
        "conditional_cols": None,
        "quantile_estimation_method": "normal_fit",
        "sample_size_thresh": 5,
        "small_sample_size_method": "std_quantiles",
        "small_sample_size_quantile": 0.98}}
    ),
     events=dict(holiday_lookup_countries=["US"]),
)

In [9]:
# Run the forecast model
forecaster = Forecaster()
result = forecaster.run_forecast_config(
     df=df,
     config=ForecastConfig(
         model_template=ModelTemplateEnum.SILVERKITE_DAILY_90.name,
         coverage=0.95,
         forecast_horizon=forecast_horizon,
         metadata_param=meta_data_params,
         evaluation_period_param=evaluation_period_param,
         model_components_param=model_components
     )
)

Fitting 92 folds for each of 3 candidates, totalling 276 fits


KeyboardInterrupt: 

In [10]:
# Get the useful fields from the forecast result
model = result.model[-1]
backtest = result.backtest
forecast = result.forecast
grid_search = result.grid_search\

# Check model coefficients / variables
# Get model summary with p-values
print(model.summary())

================================ Model Summary =================================

Number of observations: 144,   Number of features: 7
Method: Lasso regression
Number of nonzero features: 5
Regularization parameter: 0.5727

Residuals:
         Min           1Q       Median           3Q          Max
      -104.8       -22.46       -3.335         15.5        146.9

         Pred_col Estimate Pr(>)_split sig. code             95%CI  Prob_nonzero
        Intercept    131.8      <2e-16       ***    (12.88, 239.1)          1.00
     events_Other   -8.946       1.000             (-90.61, 66.45)          0.83
              ct1       0.       1.000           (-1310.0, 1330.0)          0.24
cp0_1950_05_20_00    199.8       0.001        ** (-1039.0, 1263.0)          0.98
cp1_1954_09_04_00    31.72       1.000             (-181.9, 263.7)          0.77
cp2_1958_12_20_00       0.       1.000             (-180.1, 229.0)          0.27
   y_avglag_1_2_3    153.7       0.011         *   (-137.5, 569.6) 

In [11]:
fig = forecast.plot()
iplot(fig)

In [12]:
fig=forecast.plot_components()
iplot(fig)

In [13]:
# Get cross-validation results
cv_results = summarize_grid_search_results(
     grid_search=grid_search,
     decimals=2,
     cv_report_metrics=None,
     column_order=[
         "rank", "mean_test", "split_test", "mean_train", "split_train",
         "mean_fit_time", "mean_score_time", "params"])
# Transposes to save space in the printed output
print(cv_results.transpose())

                         0
rank_test_MAPE           1
mean_test_MAPE       11.55
split_test_MAPE   (11.55,)
mean_train_MAPE       9.28
split_train_MAPE   (9.28,)
mean_fit_time         0.63
mean_score_time       0.74
params                  []


In [14]:
# Check historical evaluation metrics (on the historical training/test set).
backtest_eval = defaultdict(list)
for metric, value in backtest.train_evaluation.items():
    backtest_eval[metric].append(value)
    backtest_eval[metric].append(backtest.test_evaluation[metric])
metrics = pd.DataFrame(backtest_eval, index=["train", "test"]).T
print(metrics)

                                                         train         test
CORR                                                  0.929092     0.542434
R2                                                    0.863211     0.270862
MSE                                                 992.394718  4461.086853
RMSE                                                 31.502297    66.791368
MAE                                                  22.349179    56.135816
MedAE                                                17.290882    51.990903
MAPE                                                  9.521339    13.233279
MedAPE                                                8.310619    11.860122
sMAPE                                                 4.656115     6.428517
Q80                                                  11.174589    24.827746
Q95                                                  11.174589    23.207665
Q99                                                  11.174589    22.775643
OutsideToler

In [15]:
fig = backtest.plot()
iplot(fig)

In [16]:
pd.DataFrame(result.backtest.test_evaluation, index=["Value"]).transpose()

,Value
CORR,0.542434
R2,0.270862
MSE,4461.086853
RMSE,66.791368
MAE,56.135816
MedAE,51.990903
MAPE,13.233279
MedAPE,11.860122
sMAPE,6.428517
Q80,24.827746


In [17]:
model = result.model
future_df = result.timeseries.make_future_dataframe(
    periods=90,
    include_history=False
)

In [18]:
model.predict(future_df)

,ts,forecast,quantile_summary,err_std,forecast_lower,forecast_upper
0,1961-01-01,458.412408,"(375.79531439065084, 541.0295023030798)",42.152353,375.795314,541.029502
1,1961-02-01,469.131694,"(386.5145999351901, 551.748787847619)",42.152353,386.514600,551.748788
2,1961-03-01,479.508975,"(396.89188123590543, 562.1260691483343)",42.152353,396.891881,562.126069
3,1961-04-01,486.678706,"(404.0616124860931, 569.295800398522)",42.152353,404.061612,569.295800
4,1961-05-01,491.709549,"(409.09245519798395, 574.3266431104129)",42.152353,409.092455,574.326643
5,1961-06-01,496.193881,"(413.57678677319234, 578.8109746856212)",42.152353,413.576787,578.810975
6,1961-07-01,499.977260,"(417.3601657570422, 582.594353669471)",42.152353,417.360166,582.594354
7,1961-08-01,503.462092,"(420.8449981270998, 586.0791860395287)",42.152353,420.844998,586.079186
8,1961-09-01,506.780399,"(424.1633054601316, 589.3974933725605)",42.152353,424.163305,589.397493
9,1961-10-01,509.906904,"(427.2898103800665, 592.5239982924954)",42.152353,427.289810,592.523998


# Changepoints

In [19]:
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector

model1 = ChangepointDetector()
res = model1.find_trend_changepoints(
     df=df,            # data df
     time_col="date",    # time column name
     value_col="qty")    # value column name
pd.DataFrame({"trend_changepoints": res["trend_changepoints"]})  # prints a dataframe showing the result

,trend_changepoints
0,1949-06-01
1,1949-08-01
2,1950-07-01
3,1953-07-01
4,1953-08-01
5,1954-08-01
6,1958-12-01


In [20]:
# specify dataset information
metadata1 = dict(
     time_col="date",  # name of the time column ("datepartition" in example above)
     value_col="qty",  # name of the value column ("macrosessions" in example above)
     freq="D"        # "H" for hourly, "D" for daily, "W" for weekly, etc.
     # Any format accepted by ``pd.date_range``
 )
 # specify changepoint parameters in model_components
model_components1 = dict(
     changepoints={
         # it's ok to provide one of ``changepoints_dict`` or ``seasonality_changepoints_dict`` by itself
         "changepoints_dict": {
             "method": "auto",
             "continuous_time_col":"ct1",
             "growth_function":"linear",
             "yearly_seasonality_order": 12,
             "regularization_strength": 0.46,
             "resample_freq": "1M",
             "potential_changepoint_n": 30,
             "no_changepoint_proportion_from_end": 0.3
         },
     },
     custom={
         "fit_algorithm_dict": {
             "fit_algorithm": "lasso"}})  

In [21]:
# Generates model config
config1 = ForecastConfig.from_dict(
     dict(
         model_template=ModelTemplateEnum.SILVERKITE_DAILY_90.name,
         forecast_horizon=90,  # forecast 3 years
         coverage=0.95,  # 95% prediction intervals
         metadata_param=metadata1,
         model_components_param=model_components1))

 # Then run with changepoint parameters
forecaster1 = Forecaster()
result1 = forecaster1.run_forecast_config(
     df=df,
     config=config1)

Fitting 1 folds for each of 1 candidates, totalling 1 fits


In [22]:
forecast1 = result1.forecast
fig = forecast1.plot()
iplot(fig)

In [23]:
fig=forecast1.plot_components()
iplot(fig)

In [24]:
# Get the useful fields from the forecast result
model1 = result1.model[-1]
backtest1 = result1.backtest
forecast1 = result1.forecast
grid_search = result1.grid_search\

# Check model coefficients / variables
# Get model summary with p-values
print(model1.summary())

================================ Model Summary =================================

Number of observations: 144,   Number of features: 28
Method: Lasso regression
Number of nonzero features: 25
Regularization parameter: 0.03539

Residuals:
         Min           1Q       Median           3Q          Max
      -21.86       -3.867       0.4936        4.893        18.97

           Pred_col Estimate Pr(>)_split sig. code                95%CI  Prob_nonzero
          Intercept    111.7      <2e-16       ***      (-465.4, 327.0)          1.00
C(month,... 13)))_2    10.61       0.630                (-105.2, 99.26)          0.76
C(month,... 13)))_3    21.85    9.17e-04       ***      (-63.04, 120.4)          0.99
C(month,... 13)))_4    1.065       0.161                (-139.0, 134.8)          0.63
C(month,... 13)))_5   -9.156       0.745                (-233.2, 450.4)          0.79
C(month,... 13)))_6    -13.7       0.083         .      (-146.3, 88.59)          0.77
C(month,... 13)))_7   -20.34 

In [25]:
backtest1 = result1.backtest
backtest_eval1 = defaultdict(list)
for metric, value in backtest1.train_evaluation.items():
    backtest_eval1[metric].append(value)
    backtest_eval1[metric].append(backtest1.test_evaluation[metric])
metrics1 = pd.DataFrame(backtest_eval1, index=["train", "test"]).T
metrics1.head()

,train,test
CORR,0.997784,0.987993
R2,0.995564,0.873565
MSE,32.182037,773.568473
RMSE,5.672921,27.813099
MAE,4.344461,24.553929


In [26]:
pd.DataFrame(result1.backtest.test_evaluation, index=["Value"]).transpose()

,Value
CORR,0.987993
R2,0.873565
MSE,773.568473
RMSE,27.813099
MAE,24.553929
MedAE,25.420445
MAPE,6.174095
MedAPE,5.488894
sMAPE,2.964961
Q80,4.910786


In [27]:
model1 = result1.model
future_df1 = result1.timeseries.make_future_dataframe(
    periods=90,
    include_history=False
)

In [28]:
model1.predict(future_df1)

,ts,forecast,quantile_summary,err_std,forecast_lower,forecast_upper
0,1961-01-01,445.333607,"(429.66472181541485, 461.0024929964301)",7.994476,429.664722,461.002493
1,1961-02-01,420.264574,"(404.59568889121795, 435.9334600722332)",7.994476,404.595689,435.933460
2,1961-03-01,472.442748,"(456.77386268153975, 488.111633862555)",7.994476,456.773863,488.111634
3,1961-04-01,479.293335,"(463.62444931394054, 494.9622204949558)",7.994476,463.624449,494.962220
4,1961-05-01,494.656855,"(478.9879690067688, 510.32574018778405)",7.994476,478.987969,510.325740
5,1961-06-01,570.216461,"(554.547575454454, 585.8853466354691)",7.994476,554.547575,585.885347
6,1961-07-01,651.852639,"(636.1837529397925, 667.5215241208076)",7.994476,636.183753,667.521524
7,1961-08-01,647.448089,"(631.7792034845876, 663.1169746656027)",7.994476,631.779203,663.116975
8,1961-09-01,546.566852,"(530.8979667702795, 562.2357379512946)",7.994476,530.897967,562.235738
9,1961-10-01,483.002910,"(467.33402469725615, 498.6717958782714)",7.994476,467.334025,498.671796
